In [1]:
import numpy as np

toint = lambda x: int.from_bytes(x, byteorder='big')

In [3]:
fin = open('data/raw/360°环绕.ncae', 'rb')
magic = fin.read(4)
assert magic == b'NCAE'
ldata = toint(fin.read(4))
fin.read(6)     # unused
short = toint(fin.read(2))
lkey = toint(fin.read(1)) - 1
assert lkey % 4 == 0
key0 = bytearray(fin.read(lkey + 1))
key = np.array(key0[:4] + key0[5:]) ^ key0[4]
data = bytearray(fin.read(ldata))
fin.close()

In [4]:
key, data

(array([ 86, 150,  34,  95, 163,  37,  94, 102, 165,  65,   0,  60],
       dtype=uint8),
 bytearray(b"\x82R\x97\xd1\xec7\x8e\xb1\xf7\'}\xb6\x8f\xf5\x9c\x8d\xd2\x1d1\x02V\xf5\xa1\x0e\xd8\xcf\xbe\xb6\xb8\x16\xf9\x84\x95\xbd\xe9\x89\x0b\x14\xd5\x89\xa1fJqm\xc8G\xe2\x9d\x8c\x88Q\x90\xa1\x9d\xbaKN\t1k"))

In [5]:
byte = 0
arr = np.arange(0x100, dtype=np.uint8)
for i in range(0x100):
    byte = arr[i] + key[i % lkey] + byte & 0xFF
    arr[i], arr[byte] = arr[byte], arr[i]
arr

C:\Users\Allen Heartcore\AppData\Local\Temp\ipykernel_37200\2803794411.py:4: RuntimeWarning: overflow encountered in ubyte_scalars
  byte = arr[i] + key[i % lkey] + byte & 0xFF


array([ 86, 226,  17, 115, 120,  53, 166, 130,  79,  31,  22,  14, 188,
       112, 143, 253,  99,  44, 231, 114, 121, 186,  60, 100, 140,  41,
        20,  24, 243, 220,  81, 137, 227,  15,  42, 126, 248,  62, 251,
       129,  76, 106,  78,  69,  50, 144, 155, 139,  96,  94, 103,  90,
       149, 145, 187,  46, 150, 119, 177, 168,  73, 148,  10,   0,  40,
       178,  82,  70, 116,  37,  75,   2,  11,  91, 170, 117, 239, 222,
        98, 141,  43, 252, 160, 214,  88, 224,  65, 195, 153, 208, 158,
       152,   6,   3, 173, 122,   5, 202,  83,  36, 236, 127, 161,  19,
       105,  18,  28,  61,  30, 132, 247, 197,  49,  38, 245, 124, 147,
       123,   7, 142, 218,  54, 221, 241, 217, 255, 169,   9, 242, 133,
       213,   4,  67,  74,  63,  57, 190, 225, 201, 216, 172, 233, 131,
       118,  77, 206,  21,  52, 238, 165,  33, 250, 203, 240, 110,  66,
       109, 249, 136, 212, 113,  87, 162, 209,  97, 215,   1, 210, 104,
       128,  92, 107, 219,  47, 101, 229, 157, 193, 163, 189, 20

In [9]:
for j in range(len(data)):
    byte = arr[(j + 1) & 0xFF]
    data[j] ^= arr[arr[(byte + j + 1) & 0xFF] + byte & 0xFF]

C:\Users\Allen Heartcore\AppData\Local\Temp\ipykernel_37200\45323989.py:3: RuntimeWarning: overflow encountered in ubyte_scalars
  data[j] ^= arr[arr[(byte + j + 1) & 0xFF] + byte & 0xFF]


In [10]:
for byte in ['%02X' % x for x in data]:
    print(byte, end=' ')
data

AB 56 4A 2D 54 B2 AA 56 CA CF 53 B2 4A 4B 9C 8D D2 1D 31 02 56 F5 A1 0E D8 CF BE B6 B8 16 F9 84 95 BD E9 89 0B 14 D5 89 A1 66 4A 71 6D C8 47 E2 9D 8C 88 51 90 A1 9D BA 4B 4E 09 31 6B 

bytearray(b'\xabVJ-T\xb2\xaaV\xca\xcfS\xb2JK\x9c\x8d\xd2\x1d1\x02V\xf5\xa1\x0e\xd8\xcf\xbe\xb6\xb8\x16\xf9\x84\x95\xbd\xe9\x89\x0b\x14\xd5\x89\xa1fJqm\xc8G\xe2\x9d\x8c\x88Q\x90\xa1\x9d\xbaKN\t1k')

In [1]:
import os
for file in os.listdir('data/processed'):
    with open('data/processed/' + file, 'rb') as fin:
        content = fin.read(16)
        print('%s\t%s' % (''.join(['%02X' % x for x in content]), file))

558B410A80201405EFF2D72FC9CA28AF	acg.bin
AB564A2D54B2AA56CACF53B22A292A4D	bass.bin
3D567D4CD655147E052D8950324B910A	bass_plus.bin
AB564A2A51B2AA56CACF53B24A4BCC29	bwmrszn.bin
6D91DB6E84201086DF65AED1A0AEC6F8	church.bin
AB564A2D54B2AA56CACF53B22A292A4D	classical.bin
AB564A2D54B2AA56CACF53B24A4BCC29	concert.bin
558B310E80201004FF72F57A110C10F9	electronic.bin
6D51DB8E832010FD9779C646511BC3AF	electronic_plus.bin
558B410A80201444EFF2D79368258457	folk.bin
558B310A80301004FF72F57A24D128E6	hiphop.bin
558B310A80301004FF72F51AD47885F9	live.bin
6D51DB72832010FD977D4607B136D65F	livehouse.bin
0D9767404E5F1CC7DBA9B48786B68AD2	live_plus.bin
6D91CB6E83301045FF65D606190869C2	live_surr.bin
7D92DD6E83300C85DFC5D7A64AD23255	nineone.bin
6559097495C515FE073089881A7AD4C6	panorama.bin
0D94693C555D03C5A592A1411A50924A	radio.bin
5DCCDD0AC2300C05E077C9751A9A8DFA	rock.bin
AB564A2D54B2AA56CACF53B24A4BCC29	stereo.bin
AB564A2D54B2AA56CACF53B24A4BCC29	stereo_crystal.bin
15986938964F1B8745243B59B2640D51	stereo_studio.